# Label each row with time remaining until end of each segment

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import datetime as dt
import time
from datetime import datetime

In [6]:
df = pds.read_csv('../../bus203_all.csv')
df.head()

,Unnamed: 0,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,0,2018-02-16T04:48:40.0000000+01:00,JourneyStartedEvent,5432,203,58.414238,15.571015,-1.000000,-1.0,NaN,1,1
1,1,2018-02-16T04:48:40.0000000+01:00,ObservedPositionEvent,5432,0,58.414238,15.571015,147.300003,0.0,NaN,1,1
2,2,2018-02-16T04:48:40.0000000+01:00,ArrivedEvent,5432,203,58.414238,15.571015,-1.000000,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,1,1
3,3,2018-02-16T04:48:41.0000000+01:00,ObservedPositionEvent,5432,0,58.414246,15.571012,147.300003,0.0,NaN,1,1
4,4,2018-02-16T04:48:42.0000000+01:00,ObservedPositionEvent,5432,0,58.414249,15.571008,147.300003,0.0,NaN,1,1


Ignore entries that are not `ObservedPositionEvent`

In [7]:
df = df[df['event'] == "ObservedPositionEvent"]

This leaves the indexes of rows untouched, reset.

In [8]:
df = df.reset_index().drop(columns=['index'])

A triple loop as it seems, but it is only to finally group rows from each individual segment from every journey. Should be linear in time, as the innermost loop will do all executions and every row is visited once. This only took ~50% of my 8GB of RAM but took 850 seconds to run.

In [14]:
ts = pds.DataFrame(np.zeros(len(df.index)), columns=list('t'))

t0 = time.time()

for j, df_j in df.groupby('journey_number'):
    for k, df_s in df_j.groupby('segment_number'):
        end_time = df_s['timestamp'].iloc[-1]
        for idx, row in df_s.iterrows():
            # The subtraction returns timedelta between the two timestamp objects
            # and total seconds convert the pandas datetime object to seconds
            ts.iloc[idx] = (end_time - row['timestamp']).total_seconds()

elapsed = time.time() - t0
print("Data processed in", elapsed, " seconds")

TypeError: unsupported operand type(s) for -: 'str' and 'str'

Add time left to dataframe `df`

In [ ]:
datta = pds.concat([df, ts], axis=1)
datta.head()

... and save it.

In [ ]:
datta.to_pickle('added_t.pkl')

Extra work to add `time_since_journey_start`

In [ ]:
data = pds.read_pickle('added_t.pkl')
df = pds.read_csv('../../bus203_all_labelled.csv')
df.head()

In [ ]:
df = df[df['event'] == "ObservedPositionEvent"]
df = df.reset_index().drop(columns=['index'])

In [ ]:
data['ttjs'] = df['time_since_journey_start']
data.head()

In [ ]:
data.to_pickle('added_t_ttjs.pkl')